# Tic Tac Toe @Edge with SageMaker Neo + Tensorflow and Reinforcement Learning (RLLib)
**SageMaker Studio Kernel**: Data Science

This exercise is based on [TicTacToe with RL Tensorflow](https://github.com/aws-samples/ml-edge-getting-started/blob/main/examples/01_TicTacToe_RL_Tensorflow/01_TicTacToeRLLIB.ipynb). But, instead of training an agent, we'll just download a pre-trained 'mediocre' agent (a TF model) and create a board to play Tic Tac Toe against it. Let's do it!

In this exercise you'll:
   - Get a pre-trained model/agent that plays TicTacToe
   - Prepare the model to compile it with Neo
   - Compile the model for the target: **X86_64**
   - Get the optimized model and run a simple game
   - Have fun!

### install dependencies

In [ ]:
!pip install dlr==1.8.0

## 1) Get the model, create a package and upload it to S3

In [ ]:
import tarfile
import sagemaker
import os
import urllib.request

model_url = 'https://spock.cloud/models/tic-tac-toe-tf.tar.gz'
urllib.request.urlretrieve(model_url, 'model.tar.gz')

sagemaker_session = sagemaker.Session()
model_name='tic-tac-toe-tf'

s3_uri = sagemaker_session.upload_data('model.tar.gz', key_prefix=f'{model_name}/model')
print(s3_uri)

## 2) Compile the model with SageMaker Neo (X86_64)

In [ ]:
import time
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sm_client = boto3.client('sagemaker')
compilation_job_name = f'{model_name}-tensorflow-{int(time.time()*1000)}'
sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': s3_uri,
        'DataInputConfig': '{"agent_x/observations": [1,10]}',
        'Framework': 'TENSORFLOW'
    },
    OutputConfig={
        'S3OutputLocation': f's3://{sagemaker_session.default_bucket()}/{model_name}-tensorflow/optimized/',
        'TargetPlatform': { 
            'Os': 'LINUX', 
            'Arch': 'X86_64',
        },
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 900 }
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)

## 3) Download the compiled model

In [ ]:
output_model_path = f's3://{sagemaker_session.default_bucket()}/{model_name}-tensorflow/optimized/model-LINUX_X86_64.tar.gz'
!aws s3 cp $output_model_path /tmp/model.tar.gz
!rm -rf model_neo && mkdir model_neo
!tar -xzvf /tmp/model.tar.gz -C model_neo

## 4) Get the game code and have fun!

In [ ]:
import os
import urllib.request
game_url='https://raw.githubusercontent.com/aws-samples/ml-edge-getting-started/main/examples/01_TicTacToe_RL_Tensorflow/tictactoe/game.py'
if not os.path.exists('game.py'):
    urllib.request.urlretrieve(game_url, 'game.py') 

In [ ]:
from game import TicTacToeGame
game = TicTacToeGame()
game.run()

#### Curious to undertand how the game was implemented?

In [ ]:
!pygmentize game.py

# Have fun! :)